<a href="https://colab.research.google.com/github/Ambrose70/Applied-Data-science/blob/main/Plotlylab_dashipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 78.1 MB/s eta 0:00:00


In [3]:
#!/usr/bin/env python3
# Falcon 9 Plotly Dash — site outcomes + payload analytics

import os
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# -----------------------------
# Load data
# -----------------------------
CSV_LOCAL = "spacex_launch_dash.csv"
CSV_REMOTE = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

if os.path.exists(CSV_LOCAL):
    spacex_df = pd.read_csv(CSV_LOCAL)
else:
    spacex_df = pd.read_csv(CSV_REMOTE)

# Normalize critical columns
spacex_df["Payload Mass (kg)"] = pd.to_numeric(spacex_df["Payload Mass (kg)"], errors="coerce")
spacex_df = spacex_df.dropna(subset=["Payload Mass (kg)", "class"])

min_payload = int(spacex_df["Payload Mass (kg)"].min())
max_payload = int(spacex_df["Payload Mass (kg)"].max())
site_names = sorted(spacex_df["Launch Site"].unique())

# -----------------------------
# App
# -----------------------------
app = dash.Dash(__name__)
server = app.server  # for deployment if needed
app.title = "SpaceX Falcon 9 Dashboard"

controls_style = {"display": "grid", "gridTemplateColumns": "1fr", "gap": "12px", "maxWidth": "420px", "margin": "0 auto"}

app.layout = html.Div(
    style={"fontFamily": "Inter, Arial, sans-serif", "padding": "14px 18px", "maxWidth": "1100px", "margin": "0 auto"},
    children=[
        html.H1("SpaceX Falcon 9 – Launch Outcomes & Payload Explorer", style={"textAlign": "center", "marginBottom": "8px"}),

        html.Div([
            html.Label("Launch Site", style={"fontWeight": 600}),
            dcc.Dropdown(
                id="site-dropdown",
                options=[{"label": "All Sites", "value": "ALL"}] + [{"label": s, "value": s} for s in site_names],
                value="ALL",
                placeholder="Select a Launch Site",
                searchable=True,
                clearable=False,
            ),
            html.Label("Payload range (kg)", style={"fontWeight": 600, "marginTop": "6px"}),
            dcc.RangeSlider(
                id="payload-slider",
                min=0, max=10000, step=1000,
                marks={0: "0", 2500: "2.5k", 5000: "5k", 7500: "7.5k", 10000: "10k"},
                value=[min_payload, max_payload],
                allowCross=False,
            ),
            html.Div(id="leader-banner", style={"fontSize": "14px", "padding": "6px 10px", "border": "1px solid #ddd",
                                                "borderRadius": "10px", "background": "#fafafa"}),
        ], style=controls_style),

        html.Div(
            style={"display": "grid", "gridTemplateColumns": "1fr 1fr", "gap": "14px", "marginTop": "14px"},
            children=[
                dcc.Graph(id="success-pie-chart", config={"displayModeBar": False}),
                dcc.Graph(id="success-payload-scatter-chart", config={"displayModeBar": False}),
            ],
        ),

        html.Div("Tip: For your slide titled “KSC LC-39A: Launch Outcomes (Success vs Failure)”, select that site in the dropdown and screenshot the pie chart.",
                 style={"fontSize": "12px", "color": "#444", "marginTop": "6px"})
    ],
)

# -----------------------------
# Helpers
# -----------------------------
def site_leader_text(df):
    rate = df.groupby("Launch Site")["class"].mean().sort_values(ascending=False)
    top_site = rate.index[0]
    top_pct = f"{rate.iloc[0]*100:.1f}%"
    return f"Highest success ratio in dataset: {top_site} ({top_pct}). Use this for your 'highest-success site' screenshot."

# -----------------------------
# Callbacks
# -----------------------------
@app.callback(Output("leader-banner", "children"), Input("payload-slider", "value"))
def update_leader(_payload_range):
    # Leader is independent of the slider; we show overall site leader for guidance
    return site_leader_text(spacex_df)

@app.callback(
    Output("success-pie-chart", "figure"),
    Input("site-dropdown", "value"),
)
def update_pie(selected_site):
    if selected_site == "ALL":
        by_site = spacex_df.groupby("Launch Site", as_index=False)["class"].sum()
        fig = px.pie(by_site, values="class", names="Launch Site",
                     title="Successful Launches by Site (ALL)",
                     hole=0.25)
    else:
        site_df = spacex_df[spacex_df["Launch Site"] == selected_site]
        counts = site_df["class"].value_counts().rename_axis("Outcome").reset_index(name="Count")
        outcome_map = {0: "Failure (0)", 1: "Success (1)"}
        counts["Outcome"] = counts["Outcome"].map(outcome_map)
        fig = px.pie(counts, values="Count", names="Outcome",
                     title=f"Launch Outcomes at {selected_site}",
                     hole=0.25)
    fig.update_traces(textposition="inside", textinfo="percent+label")
    return fig

@app.callback(
    Output("success-payload-scatter-chart", "figure"),
    [Input("site-dropdown", "value"), Input("payload-slider", "value")],
)
def update_scatter(selected_site, payload_range):
    lo, hi = payload_range
    filtered = spacex_df[(spacex_df["Payload Mass (kg)"] >= lo) & (spacex_df["Payload Mass (kg)"] <= hi)]
    if selected_site != "ALL":
        filtered = filtered[filtered["Launch Site"] == selected_site]
        title = f"Payload vs Outcome — {selected_site} (Payload {lo}-{hi} kg)"
    else:
        title = f"Payload vs Outcome — ALL Sites (Payload {lo}-{hi} kg)"

    color_col = "Booster Version Category" if "Booster Version Category" in filtered.columns else None
    fig = px.scatter(
        filtered, x="Payload Mass (kg)", y="class",
        color=color_col, hover_data=["Launch Site", "Booster Version"] if "Booster Version" in filtered.columns else ["Launch Site"],
        title=title
    )
    fig.update_yaxes(tickmode="array", tickvals=[0, 1], ticktext=["Failure (0)", "Success (1)"])
    return fig

# -----------------------------
# Run
# -----------------------------
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8050, debug=True)


<IPython.core.display.Javascript object>